In [2]:
from google.colab import files
uploaded = files.upload()

Saving Suppy_Chain_Shipment_Data.csv to Suppy_Chain_Shipment_Data.csv


In [23]:
import pandas as pd

df = pd.read_csv('Suppy_Chain_Shipment_Data.csv')

display(df)
df.describe()

,id,project code,pq #,po / so #,asn/dn #,country,managed by,fulfill via,vendor inco term,shipment mode,...,unit of measure (per pack),line item quantity,line item value,pack price,unit price,manufacturing site,first line designation,weight (kilograms),freight cost (usd),line item insurance (usd)
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,30,19,551.00,29.00,0.97,Ranbaxy Fine Chemicals LTD,True,13,780.34,NaN
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,240,1000,6200.00,6.20,0.03,"Aurobindo Unit III, India",True,358,4521.5,NaN
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,100,500,40000.00,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,True,171,1653.78,NaN
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,31920,127360.80,3.99,0.07,"Ranbaxy, Paonta Shahib, India",True,1855,16007.06,NaN
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,38000,121600.00,3.20,0.05,"Aurobindo Unit III, India",True,7590,45450.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10319,86818,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,166571,599655.60,3.60,0.06,"Mylan, H-12 & H-13, India",False,See DN-4307 (ID#:83920),See DN-4307 (ID#:83920),705.79
10320,86819,104-CI-T30,FPQ-15259,SO-50102,DN-4313,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,Truck,...,60,21072,137389.44,6.52,0.11,Hetero Unit III Hyderabad IN,False,See DN-4313 (ID#:83921),See DN-4313 (ID#:83921),161.71
10321,86821,110-ZM-T30,FPQ-14784,SO-49600,DN-4316,Zambia,PMO - US,From RDC,N/A - From RDC,Truck,...,30,514526,5140114.74,9.99,0.33,Cipla Ltd A-42 MIDC Mahar. IN,False,Weight Captured Separately,Freight Included in Commodity Cost,5284.04
10322,86822,200-ZW-T30,FPQ-16523,SO-51680,DN-4334,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,17465,113871.80,6.52,0.11,Mylan (formerly Matrix) Nashik,True,1392,Freight Included in Commodity Cost,134.03


,id,unit of measure (per pack),line item quantity,line item value,pack price,unit price,line item insurance (usd)
count,10324.000000,10324.000000,10324.000000,1.032400e+04,10324.000000,10324.000000,10037.000000
mean,51098.968229,77.990895,18332.534870,1.576506e+05,21.910241,0.611701,240.117626
std,31944.332496,76.579764,40035.302961,3.452921e+05,45.609223,3.275808,500.190568
min,1.000000,1.000000,1.000000,0.000000e+00,0.000000,0.000000,0.000000
25%,12795.750000,30.000000,408.000000,4.314593e+03,4.120000,0.080000,6.510000
50%,57540.500000,60.000000,3000.000000,3.047147e+04,9.300000,0.160000,47.040000
75%,83648.250000,90.000000,17039.750000,1.664471e+05,23.592500,0.470000,252.400000
max,86823.000000,1000.000000,619999.000000,5.951990e+06,1345.640000,238.650000,7708.440000


In [24]:
import re

# Hàm xử lý: nếu giá trị dạng 'See ... (ID#:n)' thì lấy giá trị từ dòng có id=n
def resolve_reference(column_name):
    def resolver(value):
        if isinstance(value, str):
            match = re.search(r'ID#:(\d+)', value)
            if match:
                ref_id = int(match.group(1))
                ref_row = df[df['id'] == ref_id]
                if not ref_row.empty:
                    return ref_row.iloc[0][column_name]
        return value
    return df[column_name].apply(resolver)

# Áp dụng cho hai cột
df['weight (kilograms)'] = resolve_reference('weight (kilograms)')
df['freight cost (usd)'] = resolve_reference('freight cost (usd)')

In [25]:
import numpy as np

# Hàm kiểm tra xem giá trị có phải số không
def is_not_numeric(val):
    try:
        float(val)
        return False
    except (ValueError, TypeError):
        return True

non_numeric_weight = df[df['weight (kilograms)'].apply(is_not_numeric) & df['weight (kilograms)'].notnull()]
weight_counts = non_numeric_weight['weight (kilograms)'].value_counts().reset_index()
weight_counts.columns = ['non_numeric_value', 'count']
weight_counts['column_name'] = 'weight (kilograms)'

non_numeric_freight = df[df['freight cost (usd)'].apply(is_not_numeric) & df['freight cost (usd)'].notnull()]
freight_counts = non_numeric_freight['freight cost (usd)'].value_counts().reset_index()
freight_counts.columns = ['non_numeric_value', 'count']
freight_counts['column_name'] = 'freight cost (usd)'

result = pd.concat([weight_counts, freight_counts], ignore_index=True)
print(result)

                    non_numeric_value  count         column_name
0          Weight Captured Separately   1633  weight (kilograms)
1  Freight Included in Commodity Cost   1442  freight cost (usd)
2                 Invoiced Separately    344  freight cost (usd)


In [28]:
df = df[~df['weight (kilograms)'].astype(str).str.strip().str.lower().eq('weight captured separately')]
df = df[~df['freight cost (usd)'].astype(str).str.strip().str.lower().eq('Freight Included in Commodity Cost')]
df = df[~df['freight cost (usd)'].astype(str).str.strip().str.lower().eq('Invoiced Separately')]
display(df)

,id,project code,pq #,po / so #,asn/dn #,country,managed by,fulfill via,vendor inco term,shipment mode,...,unit of measure (per pack),line item quantity,line item value,pack price,unit price,manufacturing site,first line designation,weight (kilograms),freight cost (usd),line item insurance (usd)
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,30,19,551.00,29.00,0.97,Ranbaxy Fine Chemicals LTD,True,13,780.34,NaN
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,240,1000,6200.00,6.20,0.03,"Aurobindo Unit III, India",True,358,4521.50,NaN
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,100,500,40000.00,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,True,171,1653.78,NaN
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,31920,127360.80,3.99,0.07,"Ranbaxy, Paonta Shahib, India",True,1855,16007.06,NaN
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,38000,121600.00,3.20,0.05,"Aurobindo Unit III, India",True,7590,45450.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317,86816,151-NG-T30,FPQ-16313,SO-51440,DN-4282,Nigeria,PMO - US,From RDC,N/A - From RDC,Air,...,30,6724,20978.88,3.12,0.10,"Strides, Bangalore, India.",False,6183,47281.56,24.69
10318,86817,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,205243,738874.80,3.60,0.06,"Cipla, Goa, India",False,25880,46111.55,869.66
10319,86818,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,166571,599655.60,3.60,0.06,"Mylan, H-12 & H-13, India",False,25880,46111.55,705.79
10320,86819,104-CI-T30,FPQ-15259,SO-50102,DN-4313,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,Truck,...,60,21072,137389.44,6.52,0.11,Hetero Unit III Hyderabad IN,False,4426,14734.92,161.71


In [29]:
df['first line designation'] = df['first line designation'].map({True: 'Yes', False: 'No'})
display(df)

,id,project code,pq #,po / so #,asn/dn #,country,managed by,fulfill via,vendor inco term,shipment mode,...,unit of measure (per pack),line item quantity,line item value,pack price,unit price,manufacturing site,first line designation,weight (kilograms),freight cost (usd),line item insurance (usd)
0,1,100-CI-T01,Pre-PQ Process,SCMS-4,ASN-8,Côte d'Ivoire,PMO - US,Direct Drop,EXW,Air,...,30,19,551.00,29.00,0.97,Ranbaxy Fine Chemicals LTD,Yes,13,780.34,NaN
1,3,108-VN-T01,Pre-PQ Process,SCMS-13,ASN-85,Vietnam,PMO - US,Direct Drop,EXW,Air,...,240,1000,6200.00,6.20,0.03,"Aurobindo Unit III, India",Yes,358,4521.50,NaN
2,4,100-CI-T01,Pre-PQ Process,SCMS-20,ASN-14,Côte d'Ivoire,PMO - US,Direct Drop,FCA,Air,...,100,500,40000.00,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,171,1653.78,NaN
3,15,108-VN-T01,Pre-PQ Process,SCMS-78,ASN-50,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,31920,127360.80,3.99,0.07,"Ranbaxy, Paonta Shahib, India",Yes,1855,16007.06,NaN
4,16,108-VN-T01,Pre-PQ Process,SCMS-81,ASN-55,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,38000,121600.00,3.20,0.05,"Aurobindo Unit III, India",Yes,7590,45450.08,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10317,86816,151-NG-T30,FPQ-16313,SO-51440,DN-4282,Nigeria,PMO - US,From RDC,N/A - From RDC,Air,...,30,6724,20978.88,3.12,0.10,"Strides, Bangalore, India.",No,6183,47281.56,24.69
10318,86817,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,205243,738874.80,3.60,0.06,"Cipla, Goa, India",No,25880,46111.55,869.66
10319,86818,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,166571,599655.60,3.60,0.06,"Mylan, H-12 & H-13, India",No,25880,46111.55,705.79
10320,86819,104-CI-T30,FPQ-15259,SO-50102,DN-4313,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,Truck,...,60,21072,137389.44,6.52,0.11,Hetero Unit III Hyderabad IN,No,4426,14734.92,161.71


In [30]:
df.drop(columns=['dosage', 'pq first sent to client date', 'po sent to vendor date'], inplace=True)

In [31]:
date_columns = ['scheduled delivery date', 'delivered to client date', 'delivery recorded date']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

<ipython-input-31-2433a62f4c20>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
<ipython-input-31-2433a62f4c20>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
<ipython-input-31-2433a62f4c20>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')


In [33]:
# Xử lý null
import missingno as msno

# Tính toán số lượng giá trị null trong mỗi cột
null_counts = df.isnull().sum()

# Tạo một DataFrame mới để hiển thị kết quả
null_df = pd.DataFrame({'Số lượng giá trị null': null_counts})

# Sắp xếp theo số lượng giá trị null giảm dần
null_df = null_df.sort_values(by='Số lượng giá trị null', ascending=False)

# Hiển thị kết quả
print(null_df)

                            Số lượng giá trị null
shipment mode                                 313
line item insurance (usd)                     259
pq #                                            0
po / so #                                       0
asn/dn #                                        0
id                                              0
country                                         0
managed by                                      0
fulfill via                                     0
vendor inco term                                0
scheduled delivery date                         0
delivered to client date                        0
delivery recorded date                          0
project code                                    0
product group                                   0
sub classification                              0
item description                                0
vendor                                          0
brand                                           0


In [35]:
df.drop_duplicates()
df = df.dropna()
df = df.reset_index(drop=True)

print(df.isnull().sum())

display(df)

id                            0
project code                  0
pq #                          0
po / so #                     0
asn/dn #                      0
country                       0
managed by                    0
fulfill via                   0
vendor inco term              0
shipment mode                 0
scheduled delivery date       0
delivered to client date      0
delivery recorded date        0
product group                 0
sub classification            0
vendor                        0
item description              0
molecule/test type            0
brand                         0
dosage form                   0
unit of measure (per pack)    0
line item quantity            0
line item value               0
pack price                    0
unit price                    0
manufacturing site            0
first line designation        0
weight (kilograms)            0
freight cost (usd)            0
line item insurance (usd)     0
dtype: int64


,id,project code,pq #,po / so #,asn/dn #,country,managed by,fulfill via,vendor inco term,shipment mode,...,unit of measure (per pack),line item quantity,line item value,pack price,unit price,manufacturing site,first line designation,weight (kilograms),freight cost (usd),line item insurance (usd)
0,81483,108-VN-T30,FPQ-10286,SCMS-156150,ASN-16914,Vietnam,PMO - US,Direct Drop,EXW,Air,...,60,525,3465.00,6.60,0.11,"Cipla, Goa, India",Yes,34,807.47,4.86
1,92,102-NG-T01,Pre-PQ Process,SCMS-592,ASN-485,Nigeria,PMO - US,Direct Drop,EXW,Air,...,100,1000,80000.00,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,341,2682.47,128.00
2,108,104-CI-T01,Pre-PQ Process,SCMS-698,ASN-727,Côte d'Ivoire,PMO - US,Direct Drop,CIP,Air,...,60,5513,140581.50,25.50,0.42,"BMS Meymac, France",Yes,2126,0.00,224.93
3,115,108-VN-T01,Pre-PQ Process,SCMS-753,ASN-781,Vietnam,PMO - US,Direct Drop,EXW,Air,...,240,1000,1920.00,1.92,0.01,"Aurobindo Unit III, India",Yes,941,4193.49,3.07
4,116,108-VN-T01,Pre-PQ Process,SCMS-759,ASN-632,Vietnam,PMO - US,Direct Drop,FCA,Air,...,120,500,41095.00,82.19,0.68,ABBVIE Ludwigshafen Germany,Yes,117,1767.38,65.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8219,86816,151-NG-T30,FPQ-16313,SO-51440,DN-4282,Nigeria,PMO - US,From RDC,N/A - From RDC,Air,...,30,6724,20978.88,3.12,0.10,"Strides, Bangalore, India.",No,6183,47281.56,24.69
8220,86817,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,205243,738874.80,3.60,0.06,"Cipla, Goa, India",No,25880,46111.55,869.66
8221,86818,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,166571,599655.60,3.60,0.06,"Mylan, H-12 & H-13, India",No,25880,46111.55,705.79
8222,86819,104-CI-T30,FPQ-15259,SO-50102,DN-4313,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,Truck,...,60,21072,137389.44,6.52,0.11,Hetero Unit III Hyderabad IN,No,4426,14734.92,161.71


In [36]:
# Sắp xếp theo cột 'id' tăng dần
df = df.sort_values(by='id').reset_index(drop=True)
display(df)

,id,project code,pq #,po / so #,asn/dn #,country,managed by,fulfill via,vendor inco term,shipment mode,...,unit of measure (per pack),line item quantity,line item value,pack price,unit price,manufacturing site,first line designation,weight (kilograms),freight cost (usd),line item insurance (usd)
0,92,102-NG-T01,Pre-PQ Process,SCMS-592,ASN-485,Nigeria,PMO - US,Direct Drop,EXW,Air,...,100,1000,80000.00,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,Yes,341,2682.47,128.00
1,108,104-CI-T01,Pre-PQ Process,SCMS-698,ASN-727,Côte d'Ivoire,PMO - US,Direct Drop,CIP,Air,...,60,5513,140581.50,25.50,0.42,"BMS Meymac, France",Yes,2126,0.00,224.93
2,115,108-VN-T01,Pre-PQ Process,SCMS-753,ASN-781,Vietnam,PMO - US,Direct Drop,EXW,Air,...,240,1000,1920.00,1.92,0.01,"Aurobindo Unit III, India",Yes,941,4193.49,3.07
3,116,108-VN-T01,Pre-PQ Process,SCMS-759,ASN-632,Vietnam,PMO - US,Direct Drop,FCA,Air,...,120,500,41095.00,82.19,0.68,ABBVIE Ludwigshafen Germany,Yes,117,1767.38,65.75
4,130,100-HT-T01,Pre-PQ Process,SCMS-10080,ASN-628,Haiti,PMO - US,Direct Drop,EXW,Air,...,100,750,53992.50,71.99,0.72,Inverness Japan,Yes,171,3518.38,86.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8219,86816,151-NG-T30,FPQ-16313,SO-51440,DN-4282,Nigeria,PMO - US,From RDC,N/A - From RDC,Air,...,30,6724,20978.88,3.12,0.10,"Strides, Bangalore, India.",No,6183,47281.56,24.69
8220,86817,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,205243,738874.80,3.60,0.06,"Cipla, Goa, India",No,25880,46111.55,869.66
8221,86818,103-ZW-T30,FPQ-15197,SO-50020,DN-4307,Zimbabwe,PMO - US,From RDC,N/A - From RDC,Truck,...,60,166571,599655.60,3.60,0.06,"Mylan, H-12 & H-13, India",No,25880,46111.55,705.79
8222,86819,104-CI-T30,FPQ-15259,SO-50102,DN-4313,Côte d'Ivoire,PMO - US,From RDC,N/A - From RDC,Truck,...,60,21072,137389.44,6.52,0.11,Hetero Unit III Hyderabad IN,No,4426,14734.92,161.71


In [ ]:
from google.colab import files

# Lưu file
df.to_csv("data_cleaned_final.csv", index=False)
files.download("data_cleaned_final.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>